In [1]:
# Use centrailzed training to compare with federated learning
epochs = 30
n_train_items = 12800

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import numpy as np


In [3]:
torch.cuda.is_available()

True

In [15]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [16]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

train_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]


# test_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(test_loader)
#         if i < n_train_items / args.batch_size
#     ]


In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [19]:
def train(args, device, model, train_dataloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.5)
        optimizer.step()
        
        
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_dataloader) * args.batch_size,
                100. * batch_idx / len(train_dataloader), loss.item()))

In [21]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        
        
        # add gussian noise before testing
        for param in model.parameters():
            param.add_(torch.from_numpy(np.random.normal(0,1.3,param.size())*args.lr).to(device))
        
        
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)*(args.test_batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader)* (args.test_batch_size),
        100. * correct / (len(test_loader)*args.test_batch_size)))

In [22]:
with torch.no_grad():
    for param in model.parameters():
        print(param.size())
        param.add_(torch.from_numpy(np.random.normal(0,0.5,param.size())).to(device))
# print(list(model.parameters())[0])


torch.Size([16, 1, 8, 8])
torch.Size([16])
torch.Size([32, 16, 4, 4])
torch.Size([32])
torch.Size([32, 512])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


In [23]:
#%%time
model = Net().to(device)
optimizer =  optim.SGD(model.parameters(), lr=args.lr)
for epoch in range(1, args.epochs + 1):
    train(args, device, model, train_dataloader, optimizer, epoch)
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/12800 (0%)]	Loss: 2.305341
Train Epoch: 1 [1920/12800 (15%)]	Loss: 2.161102
Train Epoch: 1 [3840/12800 (30%)]	Loss: 1.921131
Train Epoch: 1 [5760/12800 (45%)]	Loss: 1.674118
Train Epoch: 1 [7680/12800 (60%)]	Loss: 1.290274
Train Epoch: 1 [9600/12800 (75%)]	Loss: 0.870902
Train Epoch: 1 [11520/12800 (90%)]	Loss: 0.858342

Test set: Average loss: 0.7868, Accuracy: 7969/10048 (79%)

Train Epoch: 2 [0/12800 (0%)]	Loss: 0.878740
Train Epoch: 2 [1920/12800 (15%)]	Loss: 0.539160
Train Epoch: 2 [3840/12800 (30%)]	Loss: 0.400589
Train Epoch: 2 [5760/12800 (45%)]	Loss: 0.536282
Train Epoch: 2 [7680/12800 (60%)]	Loss: 0.480185
Train Epoch: 2 [9600/12800 (75%)]	Loss: 0.329537
Train Epoch: 2 [11520/12800 (90%)]	Loss: 0.574379

Test set: Average loss: 0.5336, Accuracy: 8441/10048 (84%)

Train Epoch: 3 [0/12800 (0%)]	Loss: 0.579692
Train Epoch: 3 [1920/12800 (15%)]	Loss: 0.293616
Train Epoch: 3 [3840/12800 (30%)]	Loss: 0.253649
Train Epoch: 3 [5760/12800 (45%)]	Loss: 0.356332
Train 